In [12]:
from pathlib import Path
import requests
import tarfile  # Unzip `.tar` file.
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.utils.data
import torch.nn.functional as F
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader

In [2]:
print(Path.cwd())

/notebooks/PyTorch-Zoo/Load Data in PyTorch


In [8]:
ROOT_DIR = Path("/notebooks")  # Your working directory.
DATASET_DIR = ROOT_DIR / "Data"  # Directory where the datasets are/should be downloaded.
CHECKPOINT_DIR = ""  # Directory where the pretrained models are saved.

DATA_DIR = DATASET_DIR / "Flowers102"
DATA_DIR.mkdir(parents=True, exist_ok=True)  # Create directory if it doesn't exist.

image_dir = DATA_DIR / "oxford-102-flowers"

train_label_path = DATA_DIR / "oxford-102-flowers/train.txt"
val_label_path = DATA_DIR / "oxford-102-flowers/valid.txt"
test_label_path = DATA_DIR / "oxford-102-flowers/test.txt"

In [5]:
if not image_dir.is_dir():
    # Download dataset
    URL = "https://www.dropbox.com/s/hqiryv0g62lp878/oxford-102-flowers.tgz?raw=1"
    file_name = "Flowers102.tgz"
    file_path = DATA_DIR / file_name  # Download file path.

    try:
        r = requests.get(URL, allow_redirects=True)
        open(file_path, 'wb').write(r.content)
        print('[Success] Dataset downloaded successfully')
    except:
        print('[Error] Dataset downloaded failed')
        
    # Unzip dataset
    zip_file_path = file_path
    extract_directory = DATA_DIR
    
    try:
        with tarfile.open(zip_file_path, "r") as tar_ref:
            tar_ref.extractall(extract_directory)
        print('[Success] Dataset extracted successfully')
    except:
        print('[Error] Dataset extracted failed')
        
    Path.unlink(file_path)  # Remove zip file.
else:
    print('[Alert] Dataset already exist')

[Success] Dataset downloaded successfully
[Success] Dataset extracted successfully


In [6]:
def create_dataset(image_dir, label_path):
    path_list = []  # Store the path of image.
    class_list = []  # Store the class name.

    with open(label_path) as file:  # Get the content of label file.
        lines = file.readlines()

    for line in lines:
        image_path, image_class = line.strip().split(' ')
        image_path = str(image_dir / image_path)  # Convert to string type.
        path_list.append(image_path)  # Store the path of images in list.
        class_list.append(int(image_class))

    return path_list, class_list

In [9]:
train_path_list, train_class_list = create_dataset(image_dir, train_label_path)
val_path_list, val_class_list = create_dataset(image_dir, val_label_path)
test_path_list, test_class_list = create_dataset(image_dir, test_label_path)

In [13]:
class Flowers102Dataset(Dataset):
    def __init__(self, image_paths, image_class, transform=None):
        self.image_paths = image_paths
        self.image_class = image_class
        self.transform = transform
        
    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        
        image_filepath = self.image_paths[idx]
        
        # For numpy array:
        # image = cv2.imread(image_filepath)
        # image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # For torch tensor:
        image = torchvision.io.read_image(image_filepath)  # uint8 type
        image = image.to(torch.float)
        
        label = self.image_class[idx]
        
        if self.transform is not None:
            image = self.transform(image)
        
        return image, label

In [14]:
img_transforms = transforms.Compose([
    transforms.CenterCrop(227),
    transforms.RandomHorizontalFlip(),
    transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
])

In [15]:
train_dataset = Flowers102Dataset(train_path_list, train_class_list, img_transforms)
val_dataset = Flowers102Dataset(val_path_list, val_class_list, img_transforms)
test_dataset = Flowers102Dataset(test_path_list, test_class_list, img_transforms)

In [16]:
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=16, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=True)

In [17]:
for X, y in train_dataloader:
    print(X.shape)  # (bath, channel, width, height)
    print(y.shape)
    break

torch.Size([16, 3, 227, 227])
torch.Size([16])


In [18]:
!pip install pytorch-lightning 

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     |████████████████████████████████| 582 kB 14.5 MB/s eta 0:00:01
     |████████████████████████████████| 408 kB 32.6 MB/s eta 0:00:01
     |████████████████████████████████| 1.3 MB 34.1 MB/s eta 0:00:01
     |████████████████████████████████| 158 kB 35.2 MB/s eta 0:00:01


In [27]:
import pytorch_lightning as pl

class AlexNet(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        self.classifier = nn.Sequential(
            nn.Dropout(p=0.3),
            nn.Linear(256 * 6 * 6, 4096),  # !
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.3),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, 102),
        )
        
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

    def training_step(self, batch, batch_idx):
        # training_step defines the train loop.
        # it is independent of forward
        x, y = batch
        # Compute prediction error.
        y_pred = self.forward(x)
        loss = nn.CrossEntropyLoss(y_pred, y)
        # Logging to TensorBoard by default
        # self.log("train_loss", loss)
        return loss

    def configure_optimizers(self):
        optimizer = optim.Adam(params=alexnet.parameters(), lr=0.0001)
        return optimizer


In [30]:
model = AlexNet()

# training
trainer = pl.Trainer(gpus=1, num_nodes=8, precision=16, limit_train_batches=0.5, strategy=None)
trainer.fit(model, train_dataloader, val_dataloader)

Using 16bit native Automatic Mixed Precision (AMP)


MisconfigurationException: `Trainer(strategy='ddp')` or `Trainer(accelerator='ddp')` is not compatible with an interactive environment. Run your code as a script, or choose one of the compatible strategies: Trainer(strategy=None|dp|tpu_spawn). In case you are spawning processes yourself, make sure to include the Trainer creation inside the worker function.